In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SemEval2016-Task6-raw-annotations-stance (1).csv')

Overall, TF-IDF is a useful tool in NLP for identifying important terms in a document or corpus abd it is used for finding vectors

In [ ]:
import socket, pickle, numpy as np
import struct
import math

In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['Stance'] = encoder.fit_transform(df['Stance'])

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import resample
import re

In [ ]:


def rnn_tfidf(df):
    # Separate the majority and minority classes
    df_majority = df[df['Stance'] == 0]
    df_minority = df[df['Stance'] == 1]

    # Resample the majority class to match the number of samples in the minority class
    df_majority_resampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)

    # Concatenate the resampled majority class with the minority class
    df_resampled = pd.concat([df_majority_resampled, df_minority])

    # Shuffle the rows of the resampled dataset
    df_resampled = df_resampled.sample(frac=1, random_state=42)
    df = df_resampled

    def clean_data(review):
        review = re.sub('[^a-zA-Z]', ' ', review)
        review = review.lower()
        return review

    df['Tweet'] = df['Tweet'].apply(clean_data)

    def remove_stop_words(review):
        stop_words = stopwords.words('english')
        review = review.split()
        review = [word for word in review if word not in stop_words]
        review = ' '.join(review)
        return review

    df['Tweet'] = df['Tweet'].apply(remove_stop_words)

    def lematize(review):
        lemmatizer = WordNetLemmatizer()
        review = review.split()
        review = [lemmatizer.lemmatize(word) for word in review]
        review = ' '.join(review)
        return review

    df['Tweet'] = df['Tweet'].apply(lematize)

    message = df['Tweet']
    stance = df['Stance']

    # create an instance of TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer(max_features=100, lowercase=True, stop_words='english', ngram_range=(1, 1))

    # fit and transform the text data into a tf-idf matrix
    tfidf_matrix = tfidf_vectorizer.fit_transform(message)

    # convert the tf-idf matrix into a numpy array
    input_data = tfidf_matrix.toarray()

    X_train, X_test , y_train, y_test = train_test_split(input_data, stance, test_size=0.20)

    max_length = 100
    vocab_size = input_data.shape[1]

    def define_rnn_model():
       model = Sequential()
       model.add(Embedding(vocab_size, 100, input_length=max_length))
       model.add(LSTM(128))
       model.add(Dense(1, activation='sigmoid'))
       model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
       return model

    # call the define_rnn_model function to create the model object
    model = define_rnn_model()

    # print the model summary
    model.summary()

    # train the model
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

    # evaluate the model
    loss, accuracy = model.evaluate(X_test,y_test)

    # print the accuracy
    print('Testing Accuracy is {} '.format(accuracy*100))

    # make predictions on the test set and print the classification report
    y_prob = model.predict(X_test)
    y_predict = (y_prob>=0.5).astype(int)
    from sklearn.metrics import classification_report
    print(classification_report(y_test,y_predict,target_names = ["Against","Favor"]))
rnn_tfidf(df)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          10000     
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 127,377
Trainable params: 127,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
578/578 [==============================] - 145s 243ms/step - loss: 0.6934 - accuracy: 0.5038 - val_loss: 0.6929 - val_accuracy: 0.5060
Epoch 2/10
578/578 [==============================] - 144s 249ms/step - loss: 0.6951 - accuracy: 0.5093 - val_loss: 0.6946 - val_accuracy: 0.4940
Epoch 3/10
578/578 [==

In [ ]:
for i in df["Target"].unique():
  dfa = df[df["Target"]==i]
  print("........",i,".........")
  rnn_tfidf(dfa)

........ Legalization of Abortion .........
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          10000     
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 127,377
Trainable params: 127,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
108/108 [==============================] - 34s 269ms/step - loss: 0.6939 - accuracy: 0.4974 - val_loss: 0.6934 - val_accuracy: 0.4918
Epoch 2/10
108/108 [==============================] - 27s 253ms/step - loss: 0.6935 - accuracy: 0.4980 - val_loss: 0.6935 -

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     Against       0.00      0.00      0.00       460
       Favor       0.50      1.00      0.66       452

    accuracy                           0.50       912
   macro avg       0.25      0.50      0.33       912
weighted avg       0.25      0.50      0.33       912

........ Feminist Movement .........
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          10000     
                                                                 
 lstm_3 (LSTM)               (None, 128)               117248    
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 127,377
Trainable params: 127,377
Non-trainable params: 0
______________

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 100)          10000     
                                                                 
 lstm_4 (LSTM)               (None, 128)               117248    
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 127,377
Trainable params: 127,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
74/74 [==============================] - 22s 241ms/step - loss: 0.6938 - accuracy: 0.5036 - val_loss: 0.6954 - val_accuracy: 0.4821
Epoch 2/10
74/74 [==============================] - 17s 234ms/step - loss: 0.6938 - accuracy: 0.4938 - val_loss: 0.6936 - val_accuracy: 0.4821
Epoch 3/10
74/74 [========